In [1]:
import pandas as pd
import os
from tkinter import messagebox

class MatchEngine:
    def __init__(self):
        self.orders_file = "delivery_orders_2026.xlsx"
        self.drivers_file = "drivers_2026.xlsx"

    def get_data(self):
        """두 엑셀 파일을 데이터프레임으로 로드"""
        if not os.path.exists(self.orders_file) or not os.path.exists(self.drivers_file):
            return None, None
        
        df_orders = pd.read_excel(self.orders_file)
        df_drivers = pd.read_excel(self.drivers_file)
        return df_orders, df_drivers

    def run_matching(self, order_id):
        """특정 주문 번호에 대해 최적의 기사를 매칭"""
        df_orders, df_drivers = self.get_data()
        
        if df_orders is None or df_drivers is None:
            return "데이터 파일이 존재하지 않습니다."

        # 1. 해당 주문 정보 추출
        order = df_orders[df_orders["인덱스"] == order_id]
        if order.empty:
            return "해당 주문을 찾을 수 없습니다."
        
        target_order = order.iloc[0]
        target_car = target_order["차종"]

        # 2. 기사 필터링 (기본 자격 검증)
        # 로직 A: 탁송보험 가입 여부 확인 ('가입'인 기사만)
        qualified_drivers = df_drivers[df_drivers["탁송보험"] == "가입"]

        # 로직 B: 차종에 따른 면허 필터링 (예시 로직)
        # 1톤 이상 혹은 대형차량일 경우 '1종 대형' 면허 기사만 필터링
        heavy_cars = ['봉고', '포터', '마이티', '카운티', '버스']
        is_heavy = any(car in target_car for car in heavy_cars)

        if is_heavy:
            qualified_drivers = qualified_drivers[qualified_drivers["면허종류"] == "1종 대형"]
        
        if qualified_drivers.empty:
            return "조건(보험/면허)을 만족하는 기사가 없습니다."

        # 3. 최적 기사 선정 (2026년형 단순 추천 로직)
        # 실제 시스템에서는 거리 데이터나 평점을 사용하지만, 여기서는 '비고'가 깨끗하고 가입된 첫 기사 추천
        best_driver = qualified_drivers.iloc[0]

        matching_result = {
            "주문번호": order_id,
            "대상차종": target_car,
            "배정기사": best_driver["이름"],
            "기사연락처": best_driver["연락처"],
            "면허종류": best_driver["면허종류"]
        }

        return matching_result

# --- 실행 예시 ---
if __name__ == "__main__":
    matcher = MatchEngine()
    
    # 1번 주문에 대해 매칭 실행
    result = matcher.run_matching(order_id=1)
    
    if isinstance(result, dict):
        print(f"🎯 매칭 성공!")
        print(f"차종 [{result['대상차종']}] -> 기사 [{result['배정기사']}] 배정 완료")
        print(f"연락처: {result['기사연락처']} ({result['면허종류']})")
    else:
        print(f"❌ 매칭 실패: {result}")

🎯 매칭 성공!
차종 [소나타] -> 기사 [장성우] 배정 완료
연락처: 010-2975-8891 (1종대형)
